In [1]:
import os
import sys
import json
import pandas as pd

sys.path.append("../helpers/")
from impresso_id import *

data_path = "data"

%load_ext autoreload
%autoreload 2

# Pre-processing

In [2]:
with (open("RB.json", "r")) as f:
    tags = json.load(f)
    
with (open(os.path.join(data_path, "via_annotations_RB.json"), "r")) as f:
    via_annotations_RB = json.load(f)

In [3]:
df = pd.DataFrame().from_dict(tags, orient="index")
print(df.shape)
df.head()

(822, 3)


,tag,date_tag,auto_tag
JDG-1826-02-16-a-p0004-1-0,stock,"11.11.2021, 12:23:16",False
JDG-1852-11-26-a-p0004-1-0,stock,"11.11.2021, 12:32:00",False
JDG-1857-05-17-a-p0002-1-0,stock,"11.11.2021, 12:24:16",False
JDG-1857-05-17-a-p0004-1-0,stock,"11.11.2021, 12:32:10",False
JDG-1857-06-17-a-p0003-1-0,stock,"11.11.2021, 12:32:11",False


In [4]:
df["pid"] = df.index.to_series().apply(lambda x: x[:-4])

df["year"] = df["pid"].apply(lambda x: get_year(x))
df["month"] = df["pid"].apply(lambda x: get_month(x))
df["day"] = df["pid"].apply(lambda x: get_day(x))
df["journal"] = df["pid"].apply(lambda x: get_journal(x))
df["edition"] = df["pid"].apply(lambda x: get_edition(x))
df["page"] = df["pid"].apply(lambda x: get_page(x))
df["meta_issue_id"] = df["pid"].apply(lambda x: get_meta_issue_id(x))

df["date_tag"] = pd.to_datetime(df["date_tag"])

for k, v in via_annotations_RB.items():
    v['filename'] = v['filename'][:-4] + '.png'
    for i, region in enumerate(list(v['regions'])):
        table_id = k+f'-{i}'
        if table_id in df.index:                
            df.loc[table_id, 'tb_coords'] = [region['shape_attributes']]
            

print(df.shape)
df.head()

(822, 12)


,tag,date_tag,auto_tag,pid,year,month,day,journal,edition,page,meta_issue_id,tb_coords
JDG-1826-02-16-a-p0004-1-0,stock,2021-11-11 12:23:16,False,JDG-1826-02-16-a-p0004,1826,2,16,JDG,a,4,JDG-1826-02-16-a,"{'name': 'rect', 'x': 660, 'y': 1502, 'width':..."
JDG-1852-11-26-a-p0004-1-0,stock,2021-11-11 12:32:00,False,JDG-1852-11-26-a-p0004,1852,11,26,JDG,a,4,JDG-1852-11-26-a,"{'name': 'rect', 'x': 76, 'y': 1076, 'width': ..."
JDG-1857-05-17-a-p0002-1-0,stock,2021-11-11 12:24:16,False,JDG-1857-05-17-a-p0002,1857,5,17,JDG,a,2,JDG-1857-05-17-a,"{'name': 'rect', 'x': 909, 'y': 915, 'width': ..."
JDG-1857-05-17-a-p0004-1-0,stock,2021-11-11 12:32:10,False,JDG-1857-05-17-a-p0004,1857,5,17,JDG,a,4,JDG-1857-05-17-a,"{'name': 'rect', 'x': 61, 'y': 93, 'width': 63..."
JDG-1857-06-17-a-p0003-1-0,stock,2021-11-11 12:32:11,False,JDG-1857-06-17-a-p0003,1857,6,17,JDG,a,3,JDG-1857-06-17-a,"{'name': 'rect', 'x': 490, 'y': 1096, 'width':..."


## Export

In [5]:
df.to_parquet(os.path.join(data_path, "RB.parquet"))